In [ ]:
from cycliclr import CyclicLR
from math import ceil

In [ ]:
def batch_generator_df_bcolz(dframe, bcolz_arr, batch_size, shuffle=True, yielder=lambda x: x):
    assert(batch_size % bcolz_arr.chunklen == 0)
    chunks_per_batch = batch_size // bcolz_arr.chunklen
    init_bounds = bcolz_arr.partitions
    if bcolz_arr.leftover_elements > 0:
        init_bounds.append((init_bounds[-1][1], len(bcolz_arr)))
    while True:
        bounds = np.random.permutation(init_bounds) if shuffle else init_bounds
        feats_chunks, scores_chunks = [], []
        def return_batch():
            feats = np.concatenate(feats_chunks, axis=0)
            scores = np.concatenate(scores_chunks, axis=0)
            feats_chunks.clear()
            scores_chunks.clear()
            return yielder((feats, scores))
        chunks = 0
        for start, stop in bounds:
            feats_chunks.append(bcolz_arr[start:stop])
            scores_chunks.append(np.array(dframe[start:stop].score, dtype='float32'))
            chunks += 1
            if chunks % chunks_per_batch == 0:
                yield return_batch()
        if len(feats_chunks) > 0:
            yield return_batch()

In [ ]:
from bcolz_array_iterator import BcolzArrayIterator

def batch_generator_bcolz(data, batch_size, shuffle=True):
    batch_generator_bcolz = BcolzArrayIterator(data, batch_size=batch_size, shuffle=shuffle)
    while True:
        feats = next(batch_generator_bcolz)
        yield feats, feats

In [ ]:
def batch_generator_df(data, batch_size, shuffle=True):
    while True:
        data_ = data.sample(frac=1) if shuffle else data
        for iteration, batch in data_.groupby(np.arange(len(data)) // batch_size):
            fens = [f.strip() for f in batch.fen.tolist()]
            feats = np.stack([get_feats(Position.from_fen(fen)) for fen in fens])
            yield feats, feats

In [ ]:
train_model.compile(keras.optimizers.SGD(lr=.01, momentum=.95, nesterov=True), 'mse')

In [ ]:
batch_size = 1024
#nvalid = len(features_sf12_valid)

x = features_sf12_sample
x_valid = features_sf12_valid_sample

# yielder = lambda f_s: (f_s[0], [f_s[0], f_s[1]]) # (feats, [feats, scores])
train_gen = batch_generator_df_bcolz(sf_scores12, x, batch_size)
valid_gen = batch_generator_df_bcolz(sf_scores12[-60000:], x_valid, batch_size, shuffle=False)

train_steps = ceil(len(x) / batch_size)
valid_steps = ceil(len(x_valid) / batch_size)

# lr_plateau = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.05, patience=5, verbose=1)
checkpoint = keras.callbacks.ModelCheckpoint('../slonik_data/autoencoder_v0_weights.{epoch:03d}-{val_loss:.6f}.h5', 
                                             monitor='val_loss', save_best_only=True, save_weights_only=True, period=1)
# stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=20, mode='auto')
clr = CyclicLR(base_lr=0., max_lr=2, step_size=train_steps*10, mode='triangular')
# clr = CyclicLR(base_lr=0, max_lr=1e-2, step_size=train_steps*5, mode='triangular')

callbacks = [clr]
# callbacks = [clr, checkpoint]
# K.set_value(train_model.optimizer.lr, 1e-3)
train_model.fit_generator(train_gen, train_steps, epochs=10, 
                          validation_data=valid_gen, validation_steps=valid_steps, callbacks=callbacks)

In [ ]:
plt.plot(clr.history['iterations'], clr.history['lr'])

In [ ]:
rng = np.s_[:60]
plt.plot(clr.history['lr'][rng], clr.history['loss'][rng])

### evaluate models

In [ ]:
batch_size = 1024 * 2
valid_steps = ceil(len(features_sf12_valid) / batch_size)
valid_gen = batch_generator_bcolz(features_sf12_valid, batch_size, shuffle=False)
#yielder=lambda f_s: (f_s[0], [f_s[0], f_s[1]]))
# autoencoder.evaluate_generator(valid_gen, steps=valid_steps)
# train_model.evaluate_generator(valid_gen, steps=valid_steps)
# train_model.metrics_names
# K.get_value(train_model.optimizer.lr)

In [ ]:
rng = slice(0,40)
dfrng = slice(rng.start+-60000, rng.stop+-60000)
fens = sf_scores12[dfrng].orig_fen
scores = sf_scores12[dfrng].score
# np.allclose(features_sf12_valid[rng], np.array([get_feats(Position.from_fen(f)) for f in fens]))
# pprint(list(zip(scores, evaluator.predict(features_sf12_valid[rng]).squeeze())))
x = np.arange(len(scores))
plt.plot(x, scores, 'r')
plt.plot(x, model.predict(features_sf12_valid[rng]).squeeze(), 'b')